# **Week 7 Correlation Analysis**
### **Jia Ni**
In this assignment, I incorporated the LA 2023 Bicycle Crashes data [TIMS](https://tims.berkeley.edu/tools/gismap/#), visualized the crash locations and density, and analyzed the relationship between crashes and racial diversity.

### **Import the libraries**

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import numpy as np

### **Census tract data**

In [ ]:
# Read file
tracts_grouped = gpd.read_file("data/tracts_grouped.geojson")
tracts_grouped. info(verbose=True, show_counts=True)

In [ ]:
race = pd.read_csv('data/2023_race_cleaned.csv', encoding = 'utf8', dtype = {'FIPS': str})
race.info(verbose=True, show_counts=True)

In [ ]:
# Identify mismatched FIPS
tracts_fips = set(tracts_grouped["FIPS"])
race_fips = set(race["FIPS"])
extra_fips = tracts_fips - race_fips
extra_fips

In [ ]:
# Remove them
tracts_cleaned = tracts_grouped[~tracts_grouped["FIPS"].isin(extra_fips)]

# Output cleaned data
tracts_cleaned.info(verbose=True, show_counts=True)

In [ ]:
tracts_cleaned. head()

In [ ]:
tracts_cleaned.to_file("data/tracts_cleaned.geojson",driver="GeoJSON")

### **Population Density**

In [ ]:
# Calculate the area of each census tract
tracts_cleaned = tracts_cleaned.to_crs(epsg=3310)
tracts_cleaned["area_sq_km"] = tracts_cleaned.geometry.area / 1e6
tracts_cleaned = tracts_cleaned.to_crs(epsg=4326)
tracts_cleaned. head()

In [ ]:
# Read file
population = pd.read_csv('data/2023_population_cleaned.csv', encoding = 'utf8', dtype = {'FIPS': str})
population. head()

In [ ]:
population["FIPS"] = population["FIPS"].str.zfill(11)
population. head()

In [ ]:
# Merge data
population = tracts_cleaned.merge(population, on="FIPS", how="left")
population = gpd.GeoDataFrame(population, geometry="geometry")
population.info(verbose=True, show_counts=True)

In [ ]:
# Calculate the population density of each census tract
population["Population density"] = population["Total population"] / population["area_sq_km"]
population. head()

### **Map of population density**

In [ ]:
# Set up the base map
bounds = population.total_bounds
minx, miny, maxx, maxy = bounds
center_lat = (miny + maxy) / 2
center_lon = (minx + maxx) / 2

m_pd = folium.Map(location=[center_lat, center_lon], tiles="cartodb positron")
m_pd.fit_bounds([[miny, minx], [maxy, maxx]])

In [ ]:
# Count the number of tracts with no population
print((population["Total population"] == 0).sum())

In [ ]:
# Split the data into two groups
null_tracts_tp = population[population["Total population"] == 0]
nonnull_tracts_tp = population[population["Total population"] != 0]

In [ ]:
# Create the histogram of the weighted average age in LA
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 8))
plt.hist(nonnull_tracts_tp["Population density"], bins=50, color="#ffda96", edgecolor='#7a6845')

plt.title("Distribution of Population Density in LA County", fontsize=14, fontweight='bold', pad = 10)
plt.xlabel("Population Density", fontsize=12, fontweight='bold', labelpad = 10)
plt.ylabel("Frequency", fontsize=12, fontweight='bold', labelpad = 10)
plt.grid(axis="y", linestyle="--", alpha=0.3)

plt.savefig("Population Density_distribution.png", dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
# Layer 1: Gray-shaded tracts with no population
folium.GeoJson(
    null_tracts_tp,
    name="Non Population",
    style_function=lambda feature: {
        "fillColor": "gray",
        "color": "white",
        "weight": 0,
        "fillOpacity": 0.7
    }
).add_to(m_pd)

In [ ]:
# Layer 2: A choropleth map for tracts with a non-zero population

# Define refined threshold scale for better visualization
threshold_scale = [
    nonnull_tracts_tp["Population density"].min(),
    nonnull_tracts_tp["Population density"].quantile(0.05),
    nonnull_tracts_tp["Population density"].quantile(0.15),
    nonnull_tracts_tp["Population density"].quantile(0.30),
    nonnull_tracts_tp["Population density"].quantile(0.50),
    nonnull_tracts_tp["Population density"].quantile(0.70),
    nonnull_tracts_tp["Population density"].quantile(0.85),
    nonnull_tracts_tp["Population density"].quantile(0.95),
    nonnull_tracts_tp["Population density"].max(),
]

folium.Choropleth(
    geo_data=nonnull_tracts_tp,
    name="Population Density",
    data=nonnull_tracts_tp,
    columns=["FIPS", "Population density"],
    key_on="feature.properties.FIPS",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0,
    legend_name="Population Density",
    nan_fill_color="transparent",
    threshold_scale=threshold_scale
).add_to(m_pd)

In [ ]:
# Layer 3: Census tract borders with tooltips showing detailed information
folium.GeoJson(
    population,
    name="Borders",
    style_function=lambda feature: {
        "fillOpacity": 0,
        "color": "white",
        "weight": 0.3
    },
    tooltip=folium.GeoJsonTooltip(fields=["FIPS", "Population density"],
                                   aliases=["Census Tract ID", "Population Density"],
                                   localize=True)
).add_to(m_pd)

In [ ]:
# Add the layer control and show the map
folium.LayerControl().add_to(m_pd)
m_pd

### **Crashes data**

In [ ]:
# Read file
crashes = pd.read_csv('data/crashes.csv', encoding = 'utf8', dtype = {'FIPS': str})
crashes.info(verbose=True, show_counts=True)

In [ ]:
from shapely.geometry import Point

In [ ]:
# Convert crash locations into point geometries
crashes["GEOMETRY"] = crashes.apply(lambda row: Point(row["POINT_X"], row["POINT_Y"]), axis=1)
crashes_geometry = gpd.GeoDataFrame(crashes, geometry="GEOMETRY", crs="EPSG:4326")
print(crashes_geometry.crs)

In [ ]:
crashes_geometry.head()

In [ ]:
# Extract
columns_to_keep = ['CASE_ID',
                   'COLLISION_DATE',
                   'COLLISION_SEVERITY',
                   'GEOMETRY']
crashes_geometry = crashes_geometry[columns_to_keep]
crashes_geometry.info()

### **Maps of bicycle crashes**

In [ ]:
# Set up the base map
m_point = folium.Map(location=[center_lat, center_lon], tiles="cartodb positron")
m_point.fit_bounds([[miny, minx], [maxy, maxx]])

In [ ]:
# Define severity color mapping
severity_colors = {
    1: "#a60e00",
    2: "#de1200",
    3: "#ff7359",
    4: "#ff983d"
}

In [ ]:
# Map visualization
for _, row in crashes_geometry.iterrows():
    severity = row.COLLISION_SEVERITY
    color = severity_colors.get(severity, "blue")

    folium.CircleMarker(
        location=[row.GEOMETRY.y, row.GEOMETRY.x],
        radius=4,
        color="transparent",
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        weight=0,
        popup=f"Case ID: {row['CASE_ID']}<br>Date: {row['COLLISION_DATE']}<br>Severity: {row['COLLISION_SEVERITY']}"
    ).add_to(m_point)

m_point

In [ ]:
from folium.plugins import HeatMap

In [ ]:
# Set up the base map
m_heat = folium.Map(location=[center_lat, center_lon], tiles="cartodb positron")
m_heat.fit_bounds([[miny, minx], [maxy, maxx]])

In [ ]:
# 
crashes_geometry["lat"] = crashes_geometry.geometry.y
crashes_geometry["lon"] = crashes_geometry.geometry.x

#
crashes_geometry["severity_weight"] = 5 - crashes_geometry["COLLISION_SEVERITY"]
crashes_geometry["severity_weight"] = crashes_geometry["severity_weight"].astype(int)
crashes_geometry.info()

In [ ]:
#
heat_data = crashes_geometry[["lat", "lon", "severity_weight"]].values.tolist()

HeatMap(
    heat_data,
    radius=20,
    blur=10,
    max_zoom=1,
    min_opacity=0.1
).add_to(m_heat)

In [ ]:
# Display
m_heat

### **Bicycle crash density analysis**

In [ ]:
# Calculate the crash counts within each census tract
crashes_within_tracts = gpd.sjoin(crashes_geometry, tracts_cleaned, how="left", predicate="within")

In [ ]:
# Ensure compatibility and recalculate
tracts_cleaned = tracts_cleaned.to_crs(epsg=4326)
crashes_within_tracts = gpd.sjoin(crashes_geometry, tracts_cleaned, how="left", predicate="within")

crash_counts = crashes_within_tracts.groupby("FIPS").size().reset_index(name="crash_count")
crash_counts.info(verbose=True, show_counts=True)

In [ ]:
# Merge data
crashes_census_tract = tracts_cleaned.merge(crash_counts, on="FIPS", how="left").fillna(0)
crashes_census_tract. head()

In [ ]:
crashes_census_tract. info(verbose=True, show_counts=True)

In [ ]:
# Merge data
crashes_census_tract = crashes_census_tract.merge(population[['FIPS', 'Total population']], on="FIPS", how="left")
crashes_census_tract. head()

In [ ]:
# Count the number of tracts with no crash
print((crashes_census_tract["crash_count"] == 0).sum())

# Count the number of tracts with no population
print((crashes_census_tract["Total population"] == 0).sum())

In [ ]:
# Calculate the crash density of each census tract
crashes_census_tract["crash_density"] = crashes_census_tract["crash_count"] / crashes_census_tract["Total population"] * 1000
crashes_census_tract.info()

In [ ]:
invalid_density = crashes_census_tract[
    crashes_census_tract["crash_density"].isna() | 
    (crashes_census_tract["crash_density"] == float("inf"))
]

invalid_density

In [ ]:
crashes_census_tract["crash_density"] = crashes_census_tract["crash_density"].replace(float("inf"), np.nan)
crashes_census_tract.info()

In [ ]:
crashes_census_tract.to_file("data/crashes_census_tract.geojson", driver="GeoJSON")

In [ ]:
# Split the data into two groups
null_tracts_c = crashes_census_tract[(crashes_census_tract["crash_density"] == 0) | (crashes_census_tract["crash_density"].isna())]
nonnull_tracts_c = crashes_census_tract[(crashes_census_tract["crash_density"] != 0) & (~crashes_census_tract["crash_density"].isna())]

print("Null tracts count:", len(null_tracts_c))
print("Non-null tracts count:", len(nonnull_tracts_c))

In [ ]:
nonnull_tracts_c.head()

In [ ]:
# Set up the base map
m_c = folium.Map(location=[center_lat, center_lon], tiles="cartodb positron")
m_c.fit_bounds([[miny, minx], [maxy, maxx]])

In [ ]:
# Create a layered map of the crash density
# Layer 1: White-shaded tracts with no crash
folium.GeoJson(
    null_tracts_c,
    name="Non Bicycle Crash",
    style_function=lambda feature: {
        "fillColor": "white",
        "color": "white",
        "weight": 0,
        "fillOpacity": 0.7
    }
).add_to(m_c)

In [ ]:
# Layer 2: A choropleth map for tracts with crashes

# Define refined threshold scale for better visualization
threshold_scale_c = [
    nonnull_tracts_c["crash_density"].min(),
    nonnull_tracts_c["crash_density"].quantile(0.05),
    nonnull_tracts_c["crash_density"].quantile(0.15),
    nonnull_tracts_c["crash_density"].quantile(0.30),
    nonnull_tracts_c["crash_density"].quantile(0.50),
    nonnull_tracts_c["crash_density"].quantile(0.70),
    nonnull_tracts_c["crash_density"].quantile(0.85),
    nonnull_tracts_c["crash_density"].quantile(0.95),
    nonnull_tracts_c["crash_density"].max(),
]

folium.Choropleth(
    geo_data=nonnull_tracts_c,
    name="Bicycle Crash Density",
    data=nonnull_tracts_c,
    columns=["FIPS", "crash_density"],
    key_on="feature.properties.FIPS",
    fill_color="Reds",
    fill_opacity=0.7,
    line_opacity=0,
    legend_name="Bicycle Crash Density",
    nan_fill_color="transparent",
    threshold_scale=threshold_scale_c
).add_to(m_c)

In [ ]:
# Layer 3: Census tract borders with tooltips showing detailed information
folium.GeoJson(
    crashes_census_tract,
    name="Borders",
    style_function=lambda feature: {
        "fillOpacity": 0,
        "color": "white",
        "weight": 0.3
    },
    tooltip=folium.GeoJsonTooltip(fields=["FIPS", "crash_count", "crash_density"],
                                   aliases=["Census Tract ID", "Crash Count", "Crash Density"],
                                   localize=True)
).add_to(m_c)

In [ ]:
# Add the layer control and show the map
folium.LayerControl().add_to(m_c)
m_c

In [ ]:
# Find the outliers of crash density
Q1 = nonnull_tracts_c["crash_density"].quantile(0.25)
Q3 = nonnull_tracts_c["crash_density"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_tract = nonnull_tracts_c[
    (nonnull_tracts_c["crash_density"] > upper_bound) | 
    (nonnull_tracts_c["crash_density"] < lower_bound)
]

normal_tract = nonnull_tracts_c[
    (nonnull_tracts_c["crash_density"] <= upper_bound) & 
    (nonnull_tracts_c["crash_density"] >= lower_bound)
]

In [ ]:
outliers_tract.to_file("data/outliers_tract.geojson", driver="GeoJSON")
normal_tract.to_file("data/normal_tract.geojson", driver="GeoJSON")

In [ ]:
# Set up the base map
m_n = folium.Map(location=[center_lat, center_lon], tiles="cartodb positron")
m_n.fit_bounds([[miny, minx], [maxy, maxx]])

In [ ]:
# Create a layered map of the crash density
# Layer 1: White-shaded tracts with no crash
folium.GeoJson(
    null_tracts_c,
    name="Non Bicycle Crash",
    style_function=lambda feature: {
        "fillColor": "white",
        "color": "white",
        "weight": 0,
        "fillOpacity": 0.7
    }
).add_to(m_n)

In [ ]:
# Layer 2: Dark red tracts with outliers
folium.GeoJson(
    outliers_tract,
    name="Outliers Tracts",
    style_function=lambda feature: {
        "fillColor": "#c4a9a9",
        "color": "white",
        "weight": 0,
        "fillOpacity": 0.5
    }
).add_to(m_n)

In [ ]:
# Layer 3: A choropleth map for tracts with crashes
threshold_scale_n = [
    normal_tract["crash_density"].min(),
    normal_tract["crash_density"].quantile(0.05),
    normal_tract["crash_density"].quantile(0.15),
    normal_tract["crash_density"].quantile(0.30),
    normal_tract["crash_density"].quantile(0.50),
    normal_tract["crash_density"].quantile(0.70),
    normal_tract["crash_density"].quantile(0.85),
    normal_tract["crash_density"].quantile(0.95),
    normal_tract["crash_density"].max(),
]
folium.Choropleth(
    geo_data=normal_tract,
    name="Bicycle Crash Density",
    data=normal_tract,
    columns=["FIPS", "crash_density"],
    key_on="feature.properties.FIPS",
    fill_color="Reds",
    fill_opacity=0.7,
    line_opacity=0,
    legend_name="Bicycle Crash Density",
    nan_fill_color="transparent"
).add_to(m_n)

In [ ]:
# Layer 4: Census tract borders with tooltips showing detailed information
folium.GeoJson(
    crashes_census_tract,
    name="Borders",
    style_function=lambda feature: {
        "fillOpacity": 0,
        "color": "white",
        "weight": 0.3
    },
    tooltip=folium.GeoJsonTooltip(fields=["FIPS", "crash_count", "crash_density"],
                                   aliases=["Census Tract ID", "Crash Count", "Crash Density"],
                                   localize=True)
).add_to(m_n)

In [ ]:
# Add the layer control and show the map
folium.LayerControl().add_to(m_n)
m_n

In [ ]:
# Save the maps to HTMLs
m_pd.save('Population Density.html')
m_heat.save('Population Density Heat Map.html')
m_point.save('Crash Point.html')
m_c.save('Crash Density.html')
m_n.save('Crash Density-Adjusted.html')

### **Correlation analysis of race diversity and bicycle crash**

In [ ]:
# Diversity index calculated in previous work
index = gpd.read_file("data/tracts_shannon_index.geojson")
index. info(verbose=True, show_counts=True)

In [ ]:
# Combine the diversity index and crash density across different census tracts
crashes_index = nonnull_tracts_c.merge(index[['FIPS', 'Shannon_Wiener_Index']], on="FIPS", how="inner")
crashes_index.head()

In [ ]:
crashes_index.to_file("data/crashes_index.geojson", driver="GeoJSON")

### **Correlation analysis of race diversity and crash count**

In [ ]:
# Visualize the correlation for all the data using a scatter plot
import plotly.express as px

fig = px.scatter(
    crashes_index,
    x="crash_count",
    y="Shannon_Wiener_Index",
    trendline="ols",
    title="Correlation between Crash Count and Shannon_Wiener_Index",
    labels={"crash_count": "Crash Count", "Shannon_Wiener_Index": "Shannon_Wiener_Index"},
    opacity=0.6
)

fig.show()

In [ ]:
# Pearson correlation analysis
import scipy.stats as stats

corr_coeff, p_value = stats.pearsonr(crashes_index["crash_count"], crashes_index["Shannon_Wiener_Index"])

print(f"Pearson Correlation Coefficient: {corr_coeff:.4f}")
print(f"P-value: {p_value:.4f}")

### **Correlation analysis of race diversity and crash rate**

In [ ]:
# Visualize the correlation for all the data using a scatter plot
import plotly.express as px

fig = px.scatter(
    crashes_index,
    x="crash_density",
    y="Shannon_Wiener_Index",
    trendline="ols",
    title="Correlation between Crash Density and Shannon-Wiener Index",
    labels={"crash_density": "Crash Density", "Shannon_Wiener_Index": "Shannon-Wiener Index"},
    opacity=0.6
)

fig.show()

In [ ]:
# Pearson correlation analysis
import scipy.stats as stats

corr_coeff, p_value = stats.pearsonr(crashes_index["crash_density"], crashes_index["Shannon_Wiener_Index"])

print(f"Pearson Correlation Coefficient: {corr_coeff:.4f}")
print(f"P-value: {p_value:.4f}")

### **Correlation analysis of race diversity and normal crash rate**

In [ ]:
# Combine the diversity index and crash density across different census tracts excluding the outliers
effective_data = normal_tract.merge(index, on="FIPS", how="inner")
effective_data = effective_data.rename(columns={'geometry_x': 'geometry'})
effective_data = effective_data.drop(columns=['geometry_y'])
effective_data.info()

In [ ]:
# Visualize the correlation excluding the outliers
import plotly.express as px

fig = px.scatter(
    effective_data,
    x="crash_density",
    y="Shannon_Wiener_Index",
    trendline="ols",
    title="Correlation between NormaCrash Density and Shannon-Wiener Index",
    labels={"crash_density": "Crash Density", "Shannon_Wiener_Index": "Shannon-Wiener Index"},
    opacity=0.6
)

fig.show()

In [ ]:
# Pearson correlation analysis
import scipy.stats as stats

corr_coeff, p_value = stats.pearsonr(effective_data["crash_density"], effective_data["Shannon_Wiener_Index"])

print(f"Pearson Correlation Coefficient: {corr_coeff:.4f}")
print(f"P-value: {p_value:.4f}")